In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


2021-09-25 20:18:53.938548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-25 20:18:53.938612: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Yolo model

In [5]:
MODEL = '/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/yolo/yolov3-face.cfg'
WEIGHT = '/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/yolo/yolov3-wider_16000.weights'

net = cv2.dnn.readNetFromDarknet(MODEL, WEIGHT)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Face detection 

In [4]:
import datetime

str(datetime.datetime.now())

'2021-09-25 11:40:19.312593'

In [5]:
import datetime

cap = cv2.VideoCapture(0) 

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()

    # frame is now the image capture by the webcam (one frame of the video)
    # cv2.imshow('Input', frame)

    ###################################################################################
                                    # Prediction face #
    # Face detection:
    IMG_WIDTH, IMG_HEIGHT = 416,416

    # Convert image to blob:
    blob = cv2.dnn.blobFromImage(frame,
                                1/255,
                                (IMG_WIDTH, IMG_HEIGHT),
                                [0,0,0],
                                True,
                                crop=False)
    # print('blob image shape: {}'.format(blob.shape))

    # Set model input:
    net.setInput(blob)

    # Get output layers:
    output_layers = net.getUnconnectedOutLayersNames()

    # Prediction!
    # outs = net.forward()
    outs = net.forward(output_layers)

    ###################################################################################
                                # Get Bounding box info #

    # frame = image
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only
    # the ones with high confidence scores. Assign the box's class label as the
    # class with the highest score.

    confidences = []
    boxes = []

    # Each frame produces 3 outs corresponding to 3 output layers
    for out in outs: # Outs chứa tập hợp những bounding box của 3 layers (52x52, 26x26, 13x13)
            # One out has multiple predictions for multiple captured objects.
        for detection in out: # trong out chứa [x,y,w,h,có object hay k, độ chắc chắn]
            confidence = detection[-1]
            # Extract position data of face area (only area with high confidence)
            if confidence > 0.5:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)

                # Find the top left point of the bounding box 
                topleft_x = center_x - width/2
                topleft_y = center_y - height/2
                confidences.append(float(confidence))
                boxes.append([topleft_x, topleft_y, width, height])

    # Perform non-maximum suppression to eliminate 
    # redundant overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    ###################################################################################
                                # Drawing BB #
        
    result = frame.copy()
    final_boxes = []
    for i in indices:
        i = i[0]
        box = boxes[i]
        final_boxes.append(box)

        # Extract position data
        tl_x = int(box[0])
        tl_y= int(box[1])
        width = int(box[2])
        height = int(box[3])

        # print (tl_x, tl_y, width, height)
        # Draw bouding box with the above measurements
        cv2.rectangle(result, 
                    (tl_x, tl_y), 
                    ((tl_x+width), (tl_y+height)), 
                    (0,0,0), 2)
        
        # Display text about confidence rate above each box
        confidence = f'{confidences[i]:.2f}'
        cv2.putText(result, confidence, (tl_x,tl_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0), 3)

    # Display text about number of detected faces on topleft corner
    cv2.putText(result, f'number of face detected: {len(final_boxes)}', (0,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0), 3)
    
    result_rs = cv2.resize(result, (640,480))

    cv2.imshow('Face Detection', result_rs)

    # Keyboard
    c = cv2.waitKey(1)
	# Break when pressing ESC
    if c == 27:
        break

    if c == 32:
        time_now = str(datetime.datetime.now())
        img_name = time_now+'.png'
        cv2.imwrite(img_name, result_rs)
        # break

cap.release()
cv2.destroyAllWindows()

In [9]:
cap.release()
cv2.destroyAllWindows()

# Take pictures

In [16]:
import cv2
import datetime

cap = cv2.VideoCapture(1)
count = 0

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()

    # frame is now the image capture by the webcam (one frame of the video)
    cv2.imshow('Input', frame)

    c = cv2.waitKey(1)
		# Break when pressing ESC
    if c == 27:
        break

    # Take picture:
    if c == 32:
        time_now = str(datetime.datetime.now())
        img_name = time_now+'.png'
        cv2.imwrite(img_name, frame)
        count = count+1
        print(f'image number: {count}')
cap.release()
cv2.destroyAllWindows()

# Crop Face

In [7]:
# got all image paths:
import glob

all_image_path = glob.glob('/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/Data/Tam/draft_photos/*.png')
all_image_path[:5]

[]

In [8]:
crop_dir = '/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/Data/Tam/crop_images'

In [9]:
# Face detection:
import os
import datetime

def face_crop(frame):
    IMG_WIDTH, IMG_HEIGHT = 416,416
    count = 0
    # Convert image to blob:
    blob = cv2.dnn.blobFromImage(frame,
                                1/255,
                                (IMG_WIDTH, IMG_HEIGHT),
                                [0,0,0],
                                True,
                                crop=False)
    # print('blob image shape: {}'.format(blob.shape))

    # Set model input:
    net.setInput(blob)

    # Get output layers:
    output_layers = net.getUnconnectedOutLayersNames()

    # Prediction!
    # outs = net.forward()
    outs = net.forward(output_layers)

    ###################################################################################
                                # Get Bounding box info #

    # frame = image
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only
    # the ones with high confidence scores. Assign the box's class label as the
    # class with the highest score.

    confidences = []
    boxes = []

    # Each frame produces 3 outs corresponding to 3 output layers
    for out in outs: # Outs chứa tập hợp những bounding box của 3 layers (52x52, 26x26, 13x13)
            # One out has multiple predictions for multiple captured objects.
        for detection in out: # trong out chứa [x,y,w,h,có object hay k, độ chắc chắn]
            confidence = detection[-1]
            # Extract position data of face area (only area with high confidence)
            if confidence > 0.5:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)

                # Find the top left point of the bounding box 
                topleft_x = center_x - width/2
                topleft_y = center_y - height/2
                confidences.append(float(confidence))
                boxes.append([topleft_x, topleft_y, width, height])

    # Perform non-maximum suppression to eliminate 
    # redundant overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    ###################################################################################
                                # Drawing BB #
        
    result = frame.copy()
    final_boxes = []
    for i in indices:
        i = i[0]
        box = boxes[i]
        final_boxes.append(box)

        # Extract position data
        tl_x = int(box[0])
        tl_y= int(box[1])
        width = int(box[2])
        height = int(box[3])

        print (tl_x, tl_y, width, height)
        l = 3
        
        # CROP IMAGE:
        crop_img = frame[tl_y-l:tl_y+height+l , tl_x-l:tl_x+width+l]
        # save crop image:
        time_now = datetime.datetime.now()
        img_name = str(time_now) + '.png'
        cv2.imwrite(os.path.join(crop_dir, img_name), crop_img)
        count += 1


In [10]:
for img_path in all_image_path:
    image = cv2.imread(img_path)
    face_crop(image)

# Face Recognition with VGG16

## Load model

In [17]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

2021-09-26 10:42:48.251459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-26 10:42:48.251508: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
IMG_SIZE = (224,224)
BATCH_SIZE = 32
SEED = 96

In [19]:
## Load model:
# VGG16
# model = tf.keras.models.load_model('/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/model/FR_Vgg16.h5')

# EfficientNetB1
# model = tf.keras.models.load_model('/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/model/EfficientNetB1.h5')

# FR_MobileNetV2
# model = tf.keras.models.load_model('/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/model/FR_MobileNetV2.h5')

# ResNet50v2
model = tf.keras.models.load_model('/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/model/ResNet50v2.h5')

# Show the model architecture
model.summary()

# We need to define its optimizer and loss function again since the h5 file
# does not contain those information :(
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

2021-09-26 10:43:02.011561: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-26 10:43:02.011614: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-26 10:43:02.011660: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (TranTran): /proc/driver/nvidia/version does not exist


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 6147      
Total params: 23,570,947
Trainable params: 6,147
Non-trainable params: 23,564,800
___________________________________________

In [20]:
# YOLO
MODEL = '/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/yolo/yolov3-face.cfg'
WEIGHT = '/home/tamtran/Documents/GitHub/ThuTamNguyen_facial_recognition/yolo/yolov3-wider_16000.weights'
net = cv2.dnn.readNetFromDarknet(MODEL, WEIGHT)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [21]:
def get_top_prediction(prediction, top, class_names):
    '''
    return a tuple have: top_confidence, top_id, top_class_names
    '''
    for i in range(top):
        top_confidence = list(np.sort(prediction)[0][::-1])[:top]
        top_id = list(np.argsort(prediction, axis=1)[0][::-1])[:top]
        top_class_names = [class_names[id] for id in top_id]

    return top_confidence, top_id, top_class_names


In [22]:

# Face detection:
import os
import datetime
class_names = ['Nguyen', 'Tam', 'Thu']

def face_prediction(frame):
    IMG_WIDTH, IMG_HEIGHT = 416,416
    count = 0
    # Convert image to blob:
    blob = cv2.dnn.blobFromImage(frame,
                                1/255,
                                (IMG_WIDTH, IMG_HEIGHT),
                                [0,0,0],
                                True,
                                crop=False)
    # Set model input:
    net.setInput(blob)

    # Get output layers:
    output_layers = net.getUnconnectedOutLayersNames()
   
    # Prediction!
    outs = net.forward(output_layers)

    ###################################################################################
                                # Get Bounding box info #

    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only
    # the ones with high confidence scores. Assign the box's class label as the
    # class with the highest score.

    confidences = []
    boxes = []

    # Each frame produces 3 outs corresponding to 3 output layers
    for out in outs: # Outs chứa tập hợp những bounding box của 3 layers (52x52, 26x26, 13x13)
            # One out has multiple predictions for multiple captured objects.
        for detection in out: # trong out chứa [x,y,w,h,có object hay k, độ chắc chắn]
            confidence = detection[-1]
            # Extract position data of face area (only area with high confidence)
            if confidence > 0.5:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)

                # Find the top left point of the bounding box 
                topleft_x = center_x - width/2
                topleft_y = center_y - height/2
                confidences.append(float(confidence))
                boxes.append([topleft_x, topleft_y, width, height])

    # Perform non-maximum suppression to eliminate 
    # redundant overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    ###################################################################################
                                # Drawing BB #
    result = frame.copy()
    final_boxes = []
    for i in indices:
        i = i[0]
        box = boxes[i]
        final_boxes.append(box)

        # Extract position data
        tl_x = int(box[0])
        tl_y= int(box[1])
        width = int(box[2])
        height = int(box[3])

        # print (tl_x, tl_y, width, height)
        
        # CROP IMAGE:
        l = 3
        crop_img = frame[tl_y-l:tl_y+height+l , tl_x-l:tl_x+width+l]
        
        # PREDICT:
        crop_img = cv2.resize(crop_img, IMG_SIZE)
        crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        crop_img = tf.expand_dims(crop_img, axis=0)
        
        prediction = model.predict(crop_img)
        
        # GET TOP PREDICTION:
        confidence, _, class_name = get_top_prediction(prediction=prediction, top=1, class_names=class_names)

        # DRAW RECTANGLE:
        offset = 5
        cv2.rectangle(frame, 
                (tl_x-offset, tl_y-offset), 
                ((tl_x+width+offset), (tl_y+height+offset)), 
                (0,0,0), 2)
        
        # DISPLAY TEXT:
        class_text = f'{class_name[0]} - {np.round(confidence[0], 2)}'
        l = 10
        cv2.putText(frame, class_text, (tl_x-l, tl_y-l), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    return frame


## Webcam

In [24]:
cap = cv2.VideoCapture(1)
count = 0

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()

    # frame is now the image capture by the webcam (one frame of the video)

    c = cv2.waitKey(1)
		# Break when pressing ESC
    if c == 27:
        break

    # Take picture:
    if c == 32:
        time_now = str(datetime.datetime.now())
        img_name = time_now+'.png'
        cv2.imwrite(img_name, frame)
        count = count+1
        print(f'image number: {count}')

    frame = face_prediction(frame)

    cv2.imshow('Input', frame)

cap.release()
cv2.destroyAllWindows()

2021-09-26 10:43:40.919252: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
